In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import csv

In [63]:
#Functions needed for cointegration test and statistical tests
limit=5000
def get_price(tick,res):
    res=str(res)
    hist=requests.get('https://ftx.com/api/markets/'+tick+'-PERP/candles?resolution='+res+'&start_time='+'0'+'&limit='+str(limit)).json()
    hist=pd.DataFrame(hist['result'])
    hist.drop(['startTime'], axis = 1, inplace=True)
    return hist

def get_time(hist):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    aux.set_index('time', inplace=True)
    return aux

def get_initime(hist):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    return aux['time'][0]

def get_time(hist,i):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    return aux['time'][i]
def coint_test(tick1,tick2,res):
    A=get_price(tick1,res)
    B=get_price(tick2,res)
    if not get_initime(B)==get_initime(A):
        print('Different initial time')
        return 
    X = np.stack((np.log10(A['close']), np.log10(B['close'])), axis=0)
    cov=np.cov(X)
    #print(cov)
    covAB=cov[0,1]
    varA=cov[0,0]
    varB=cov[1,1]
    cor=covAB/(np.sqrt(varA*varB))
    lAB=covAB/varB #res=log(A)-lAB*log(B)
    lBA=covAB/varA #res=log(B)-lBA*log(A)
    spreadAB=np.log10(A['close'])-lAB*np.log10(B['close'])
    muAB=np.mean(spreadAB)
    spreadBA=np.log10(B['close'])-lBA*np.log10(A['close'])
    muBA=np.mean(spreadBA)
    return cor,lAB,lBA,muAB,muBA,spreadAB,spreadBA

def print_spread(tick1,tick2,res):
    aux=coint_test(tick1,tick2,res)
    if aux[1]>aux[2]:
        print('We are gonna use A-lAB*B with correlation='+str(aux[0])+' and lambda_AB='+str(aux[1]))
        plt.plot(aux[5]-aux[3])
        return aux[5]
    else :
        print('We are gonna use B-lBA*A with correlation='+str(aux[0])+' and lambda_BA='+str(aux[2]))
        plt.plot(aux[6]-aux[4])
        return aux[6]
def get_spread(tick1,tick2,res):
    aux=coint_test(tick1,tick2,res)
    if aux[1]>aux[2]:
        return aux[5],aux[0],np.mean(aux[5]),aux[1],aux[2]
    else :
        return aux[6],aux[0],np.mean(aux[6]),aux[1],aux[2]

In [55]:
pdf= pd.read_csv('pair_ro.csv',header=0)

In [111]:
#Filter the ro values
res=3600
ro_limit=0.9
pdf_f=pdf[pdf['ro']>ro_limit]

In [112]:
len(pdf_f['Ticker1'])

274

In [113]:
pair_opt=[]
for i in range(len(pdf_f['Ticker1'])):
    tick1=pdf_f['Ticker1'][i]
    tick2=pdf_f['Ticker2'][i]
    print(i)
    aux=get_spread(tick1,tick2,res)
    diff=abs(aux[2]-aux[0][len(aux[0])-1])
    pair_opt.append((tick1,tick2,aux[1],diff,aux[3],aux[4]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273


In [114]:
po_df=pd.DataFrame(pair_opt)
po_df.columns = ['Ticker1', 'Ticker2','ro','diff','lAB','lBA']
po_df=po_df.sort_values(by=['diff'],ascending=False)

In [115]:
po_df

,Ticker1,Ticker2,ro,diff,lAB,lBA
245,ADA,MAPS,0.908979,0.209903,0.607450,1.360181
269,LINK,REEF,0.886777,0.189379,0.699481,1.124224
43,ALGO,ATOM,0.943325,0.187405,0.699473,1.272189
39,ETC,ICP,0.939632,0.187217,0.636088,1.388028
202,BAND,LINK,0.894311,0.179159,0.964639,0.829111
...,...,...,...,...,...,...
174,ONT,PUNDIX,0.919669,0.000639,0.799572,1.057806
38,FTT,ORBS,0.949971,0.000467,1.021045,0.883845
9,BNB,SHIT,0.968630,0.000409,0.961708,0.975602
12,FLM,ONT,0.964687,0.000405,1.160643,0.801815


In [117]:
po_df.to_csv('pairs_optimal.csv')